In [1]:
import psycopg2
import pandas as pd
import hsml
from constants import *

In [2]:
connection = psycopg2.connect(**DB_CONFIG)
cursor = connection.cursor()

tables = [
    'city_weather',
    'drivers_table',
    'routes_table',
    'routes_weather',
    'traffic_table',
    'truck_schedule_table',
    'trucks_table'
]

In [3]:
def get_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    return rows, columns

In [4]:
class HopsworksFeatureStoreManager:
    def __init__(self, project_name, feature_store_name, api_key, host, port=443):
        """
        Initialize the connection to Hopsworks Feature Store.
        """
        self.project_name = project_name
        self.feature_store_name = feature_store_name
        self.api_key = api_key
        self.host = host
        self.port = port
        self.fs = None

    def connect_to_hopsworks(self):
        """
        Connect to Hopsworks and get the feature store.
        """
        print(self.port)
        connection = hsml.connection(
            host=self.host,
            port=self.port,
            project=self.project_name,
            api_key_value=self.api_key
        )
        self.fs = connection.get_feature_store(self.feature_store_name, self.project_name)

    def create_feature_group(self, columns, group_name, primary_key, version=1, description=""):
        feature_group = self.fs.create_feature_group(
            name=group_name,
            version=version,
            description=description,
            primary_key=primary_key,
            online_enabled=True
        )
        return feature_group

    def insert_data_into_feature_group(self, feature_group, data):
        feature_group.insert(data)

def ingest_sql_data_to_hopsworks(tables, cursor, hopsworks_manager):
    for table_name in tables:
        rows, columns = get_data_from_table(table_name)

        primary_key = ['id'] if 'id' in columns else columns[:1]
        feature_group = hopsworks_manager.create_feature_group(
            columns=columns,
            group_name=table_name,
            primary_key=primary_key,
            description=f"Feature group for {table_name}"
        )

        for row in rows:
            hopsworks_manager.insert_data_into_feature_group(feature_group, row)

In [5]:
project_name = "Truck_Delay2"
feature_store_name = "your_feature_store_name"
# Insert the API key here
host = "https://c.app.hopsworks.ai/p/1022104"  # Hopsworks server URL or IP address
port = 443
hopsworks_manager = HopsworksFeatureStoreManager(
    project_name=project_name,
    feature_store_name=feature_store_name,
    api_key=HOPSWORK_API_KEY,
    host=host
    port =port
)

hopsworks_manager.connect_to_hopsworks()

# Ingest data from SQL to Hopsworks
# ingest_sql_data_to_hopsworks(tables, cursor, hopsworks_manager)

443


ValueError: Invalid port ''.

In [4]:
import os
import dotenv
import joblib

filename = '../Data/DataFrame/dataframes_dict.joblib'
loaded_dataframes_dict = joblib.load(filename)

# Access the loaded DataFrames (for verification)
# print(loaded_dataframes_dict['df1'])
# print(loaded_dataframes_dict['df2'])
loaded_dataframes_dict['drivers_table']

,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,d9f30553-6,Daniel Marks,male,47,5,proactive,7,42302347,62.22
1,82de7bb8-2,Clifford Carr,male,47,14,proactive,4,27867488,60.89
2,7e789842-4,Terry Faulkner MD,male,41,9,conservative,2,13927774,53.67
3,b2555587-8,Brendan Jacobs,male,44,10,proactive,2,69577118,59.82
4,b2e58421-d,Vincent Davis,male,41,10,proactive,7,28650047,62.65
...,...,...,...,...,...,...,...,...,...
1295,57f77e99-8,Robert Johnson,male,60,26,proactive,4,17792974,60.64
1296,e64c4059-8,Patrick Hernandez,male,46,20,proactive,8,17248432,58.42
1297,7e6a0cf5-1,Richard Ortiz,male,49,10,conservative,7,21865328,48.29
1298,c4a966b0-b,Peter Rodriguez,male,49,5,conservative,8,33564827,43.14
